This is a notebook that has the simplest implementation of KAN-ODEs. It shows how to generate the data, define a KAN then pass it to a NeuralODE, then shows how to train it. It also introduces the `ProgressBar.jl` package. There's no plotting or additional data saved.

In [48]:
using Random, Lux, LinearAlgebra
using NNlib, ConcreteStructs, WeightInitializers, ChainRulesCore
using ComponentArrays
using BenchmarkTools
using OrdinaryDiffEq, DiffEqFlux, ForwardDiff
using Flux: Adam, mae, update!
using Flux
using Optimisers
using MAT
using ProgressBars
using Zygote: gradient as Zgrad

# Load the KAN package from https://github.com/vpuri3/KolmogorovArnold.jl
include("src/KolmogorovArnold.jl")
using .KolmogorovArnold
#load the activation function getter (written for this project, see the corresponding script):
include("Activation_getter.jl")

activation_getter (generic function with 1 method)

# Defining the ODE and Generating Data

First we generate data from the ODE 

$$ \begin{cases}
x' &= \alpha x - \beta x y \\
y' &= \gamma x y - \delta y \end{cases}$$

Note we'll define the function using the inplace style as this saves allocating space in memory, speeding up the execution. 

In [34]:
function lotka!(du, u, p, t)
    α, β, γ, δ = p
    du[1] = α * u[1] - β * u[2] * u[1]
    du[2] = γ * u[1] * u[2] - δ * u[2]
end

lotka! (generic function with 1 method)

Next we simulate the ODE and generate the data by defining a object of type [ODEProblem](https://docs.sciml.ai/DiffEqDocs/stable/types/ode_types/) and calling [solve](https://docs.sciml.ai/DiffEqDocs/stable/basics/common_solver_opts/#CommonSolve.solve-Tuple%7BSciMLBase.AbstractDEProblem,%20Vararg%7BAny%7D%7D) which are part of the `DifferentialEquations.jl` package.

In [35]:
#data generation parameters
timestep=0.1
rng = Random.default_rng()
Random.seed!(rng, 0)
tspan = (0.0, 14)
tspan_train=(0.0, 3.5)
u0 = [1, 1]
#Parameters of the ODE [α, β, γ, δ]
p_ = Float32[1.5, 1, 1, 3]
prob = ODEProblem(lotka!, u0, tspan, p_)

#generate training data, split into train/test
solution = solve(prob, Tsit5(), abstol = 1e-12, reltol = 1e-12, saveat = timestep)
end_index=Int64(floor(length(solution.t)*tspan_train[2]/tspan[2]))
t = solution.t #full dataset
t_train=t[1:end_index] #training cut
X = Array(solution)
Xn = deepcopy(X);

# Defining the KAN

Here we define the architecture of the KAN. Using the `KolmogorovArnold.jl` package, we can call Lux layers that are defined there to save us the work of defining them by ourselves. We may choose 
* which basis functions we would like (defined in `src/utils.jl`)
* which normalizer functions we would like (defined in `src/utils.jl`)
* the grid size the basis functions will use
* the number of layers to our KAN
* the width of the layers

In [36]:
basis_func = rbf      # rbf, rswaf
normalizer = tanh_fast # sigmoid(_fast), tanh(_fast), softsign 
grid_size=5 #Grid size for the activation functions 
##Not sure what this is? It seems like this normalizes the inputs 
##to be between -1,1,/0,1 but i dont quite see for sure where.
num_layers=2 #number of layers in the KAN 
layer_width=10 #Width of each layer in the KAN (number of activation functions )

kan1 = Lux.Chain(
    KDense( 2, layer_width, grid_size; use_base_act = true, basis_func, normalizer),
    KDense(layer_width,  2, grid_size; use_base_act = true, basis_func, normalizer),
)
pM , stM  = Lux.setup(rng, kan1) #Assign parameters and the state to memory


pM_axis = getaxes(ComponentArray(pM))
pM_data = getdata(ComponentArray(pM))
p = (deepcopy(pM_data))./1e5 ;

# Preparing to train

## Construct a `NeuralODE` 

When we defined our KAN we constructed a model in Lux. With this type, we can easily pass it to the NeuralODE package for training. In order to train our model, we need to 
* Construct a `NeuralODE`
* Define a loss function
* Choose an optimizer
* Choose an algorithm for constructing the gradient.

In addition to training the model, we also want to see how well the model is predicting the future state of the ODE. To do this, we will instantiate a second `NeuralODE` called `train_node_test` which will have a timespan over the test set but will be built with the same `kan1`. This will allow us to easily see how our model performs over the test set.

In [37]:
train_node      = NeuralODE(kan1, tspan_train, Tsit5(), saveat = t_train); 
train_node_test = NeuralODE(kan1, tspan, Tsit5(), saveat = t); #only difference is the time span

function predict(p)
    Array(train_node(u0, p, stM)[1])
end
function predict_test(p)
    Array(train_node_test(u0, p, stM)[1])
end

predict_test (generic function with 1 method)

## Define a loss function

In order to train our model, we will need to specify a loss function. Probably the simplest loss function is the mean square error (MSE)
$$\mathcal{L}_1(\theta) = MSE(u^{\text{KAN}}(t, \theta), u^{\text{obs}}(t)) = \frac{1}{N}\sum_{i=1}^N \lVert u^{\text{KAN}}(t_i, \theta) - u^{\text{obs}}(t_i) \rVert^2 $$ 
which are implemented as functions of the parameter `p` like so


In [38]:
function loss_train(p)
    mean(abs2, Xn[:, 1:end_index].- predict(ComponentArray(p,pM_axis)))
end
function loss_test(p)
    mean(abs2, Xn .- predict_test(ComponentArray(p,pM_axis)))
end

loss_test (generic function with 1 method)

However, in KAN-ODEs paper, we may also want to include a term to encourage sparcity in our model. One way we can do this is to add an $l1$ norm term to our loss 
$$\mathcal{L}_2(\theta) = \frac{1}{N}\sum_{i=1}^N \| u^{\text{KAN}}(t_i, \theta) - u^{\text{obs}}(t_i) \|^2  + \gamma_{sp} | \theta |_1 $$
which introduces a sparcity hyperparameter $\gamma_{sp}$ which we may control. We'll add the sparcity term as a function `reg_loss` and add it to the result of `loss_train` for our total loss function `loss`.

In [39]:
#regularization loss 
function reg_loss(p, reg=1.0)
    l1_temp=(abs.(p))
    activation_loss=sum(l1_temp)
    activation_loss*reg
end

#overall loss is the sum of trhe training loss and the sparcity regularization loss
function loss(p)
    loss_MSE=mean(abs2, Xn[:, 1:end_index].- predict(ComponentArray(p,pM_axis)))
    loss_MSE + reg_loss(p, 5e-4)
end

loss (generic function with 1 method)

## Choosing an Optimizer
The `Flux.jl` has an assortment of optimizers to choose from. Here we will choose `Adam` with a learning rate of `5e-4`

In [40]:
optimizer = Flux.Adam(5e-4)

Adam(0.0005, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

## Choosing a Gradient Algorithm

We are also able to choose which algorithm we would like to use to calculate the gradient of our loss function. We'll use the `Zgrad` alogirthm and include this explicitly when we do the training loop. (TODO: Why?)

# Training Loop

At this stage we have everything we need to start the training loop. 

We'll want to see how far along we are in our training algorithm, so we'll want to make use of the `ProgressBars.jl` package. Passing a range into `ProgressBar` creates an object with Julia can iterate over, which also includes some extra functionality. By default, a progress bar will now appear in our terminal showing how far along our for loop is, as well as the rate at which it is iterating. We can also include additional information such as the current loss with `set_description`.

In the training loop, we only do 3 actions
1. Calculate the gradient using `Zgrad`
2. Update the parameters of the model
3. Calculate the current loss and test loss and print them to the `ProgressBar`

In [ ]:
using Printf
# TRAINING
N_iter = 100 #Number of training iterations

iters=ProgressBar(1:N_iter)
for i in iters
    
    # gradient computation
    grad = Zgrad(loss, p)[1]

    #model update
    update!(optimizer, p, grad)

    #loss metrics
    loss_curr=deepcopy(loss_train(p))
    loss_curr_test=deepcopy(loss_test(p))
    set_description(iters, string("Loss:", @sprintf("%.2f", loss_curr), "|",
                            "Train_Loss:", @sprintf("%.2f", loss_curr_test), "|"))
end


0.0%┣                                              ┫ 0/100 [00:00<00:-1, -0s/it]
Loss:2.98|Train_Loss:9.45| 3.0%┣▋                  ┫ 3/100 [00:00<00:03, 30it/s]
Loss:2.97|Train_Loss:9.42| 6.0%┣█▏                 ┫ 6/100 [00:00<00:02, 39it/s]
Loss:2.96|Train_Loss:9.42| 9.0%┣█▊                 ┫ 9/100 [00:00<00:02, 43it/s]
Loss:2.94|Train_Loss:9.47| 12.0%┣██               ┫ 12/100 [00:00<00:02, 46it/s]
Loss:2.93|Train_Loss:9.55| 15.0%┣██▌              ┫ 15/100 [00:00<00:02, 46it/s]
Loss:2.92|Train_Loss:9.67| 18.0%┣███              ┫ 18/100 [00:00<00:02, 48it/s]
Loss:2.90|Train_Loss:9.83| 21.0%┣███▋             ┫ 21/100 [00:00<00:02, 48it/s]
Loss:2.89|Train_Loss:10.04| 24.0%┣███▉            ┫ 24/100 [00:00<00:02, 48it/s]
Loss:2.87|Train_Loss:10.30| 27.0%┣████▎           ┫ 27/100 [00:01<00:01, 49it/s]
Loss:2.86|Train_Loss:10.61| 30.0%┣████▉           ┫ 30/100 [00:01<00:01, 49it/s]
Loss:2.84|Train_Loss:10.97| 33.0%┣█████▎          ┫ 33/100 [00:01<00:01, 50it/s]
Loss:2.82|Train_Loss:11.39| 